In [ ]:
# !pip install gensim
# !pip install nltk

In [ ]:
import numpy as np
import pandas as pd
import os

import pickle
from sklearn.linear_model import LogisticRegression
import sklearn.pipeline
from cross_validation import train_models_and_calc_scores_for_n_fold_cv
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# data_dir='./data_reviews'

# # read in training data
# x_train_df = pd.read_csv(os.path.join(data_dir, 'x_train.csv'))
# y_train_df = pd.read_csv(os.path.join(data_dir, 'y_train.csv'))

# # create a list of all reviews
# x_text_list = x_train_df.values[:,1]
# x_text_list = x_text_list.tolist()

# tokens = [word_tokenize(review.lower()) for review in x_text_list]

# word_2_vec = Word2Vec(tokens, vector_size=100, window=5, min_count=1, workers=4)
# print(x_text_list)

In [ ]:
# # takes average of vector for all words in a sentence
# def average_word_vectors(words, model, vocabulary, num_features):
#     feature_vector = np.zeros((num_features,), dtype="float64")
#     nwords = 0.
    
#     for word in words:
#         if word in vocabulary: 
#             nwords = nwords + 1.
#             feature_vector = np.add(feature_vector, model.wv[word])
    
#     if nwords:
#         feature_vector = np.divide(feature_vector, nwords)
        
#     return feature_vector

# # list of vectors
# feature_vectors = [average_word_vectors(review, word_2_vec, word_2_vec.wv.key_to_index, 100) for review in x_text_list]
# x_train_matrix = np.stack(feature_vectors)

In [ ]:
data_dir='./data_reviews'

# read in training data
x_train_df = pd.read_csv(os.path.join(data_dir, 'x_train.csv'))
y_train_df = pd.read_csv(os.path.join(data_dir, 'y_train.csv'))

# create a list of all reviews
x_text_list = x_train_df.values[:,1]
x_text_list = x_text_list.tolist()

vectorizer = TfidfVectorizer(stop_words="english")

# fit_transform will make its own vocabular from the text
x = vectorizer.fit_transform(x_text_list)  
x_train_matrix = x.toarray()

# export vocabulary
vocabulary = vectorizer.vocabulary_
with open('vocab2.pkl','wb') as f:
    pickle.dump(vocabulary,f)

In [ ]:
# # export vocabulary
# with open('word2vec_model.pkl','wb') as f:
#     pickle.dump(word_2_vec,f)

In [ ]:

clf = RandomForestClassifier()

# C is hyperparam for regularization
param_grid = {}

# by default,um folds is 5
search = HalvingGridSearchCV(clf, param_grid,cv=5, random_state=0).fit(x_train_matrix, y_train_df)

In [ ]:
print(search.best_params_)
#print(search.cv_results_['split0_test_score'])
#print(search.cv_results_['params'])

# indices = []
# for index, dict in enumerate(search.cv_results_['params']):
#     if dict['tol'] == search.best_params_['tol']:
#         indices.append(index)

# print(indices)
# for idx in indices:
#     print(search.cv_results_['params'][idx])

#indices = indices[0:17]
# test_scores = []
# train_scores = []
# test_split0, train_split0, test_split1, train_split1, test_split2, train_split2, test_split3, train_split3, test_split4, train_split4 = ([] for i in range(10))
# for idx in indices:
#     test_scores.append(search.cv_results_['mean_test_score'][idx])
#     train_scores.append(search.cv_results_['mean_train_score'][idx])
#     test_split0.append(search.cv_results_['split0_test_score'][idx])
#     train_split0.append(search.cv_results_['split0_train_score'][idx])
#     test_split1.append(search.cv_results_['split1_test_score'][idx])
#     train_split1.append(search.cv_results_['split1_train_score'][idx])
#     test_split2.append(search.cv_results_['split2_test_score'][idx])
#     train_split2.append(search.cv_results_['split2_train_score'][idx])
#     test_split3.append(search.cv_results_['split3_test_score'][idx])
#     train_split3.append(search.cv_results_['split3_train_score'][idx])
#     test_split4.append(search.cv_results_['split4_test_score'][idx])
#     train_split4.append(search.cv_results_['split4_train_score'][idx])

# print(test_scores)
# print(train_scores)

In [ ]:
model = RandomForestClassifier()
y_train_labels = y_train_df
model.fit(x_train_matrix, y_train_labels.values.ravel())

# save classifiers to pkl files
with open('prob2model.pkl','wb') as f:
    pickle.dump(model,f)

In [ ]:
x =  np.logspace(-10,6,17)
plt.plot(x, train_scores, label="train scores")
plt.plot(x, test_scores, label="test scores")
plt.scatter(x, test_split0, color="orange")
plt.scatter(x, test_split1, color="orange")
plt.scatter(x, test_split2, color="orange")
plt.scatter(x, test_split3, color="orange")
plt.scatter(x, test_split4, color="orange")
plt.scatter(x, train_split0, color="blue")
plt.scatter(x, train_split1, color="blue")
plt.scatter(x, train_split2, color="blue")
plt.scatter(x, train_split3, color="blue")
plt.scatter(x, train_split4, color="blue")
plt.legend()
plt.title("Hyperparameter Search: scores vs. C")
plt.xlabel("Inverse Regularization Strength: C")
plt.ylabel("Score")
plt.xscale('log')
plt.show()

In [ ]:
# train logistic regression with the selected best params
classifier = LogisticRegression(C=search.best_params_["C"], tol=search.best_params_["tol"], solver=search.best_params_["solver"]) 
classifier.fit(x_train_NV, y_train_df.values.ravel())

# save classifiers to pkl files
with open('testing2.pkl','wb') as f:
    pickle.dump(classifier,f)

In [ ]:

y_train_df = np.array(y_train_df)
y_train_df.shape
y_train_df = y_train_df.reshape((2400,))
print(y_train_df.shape)

In [ ]:
print(x_train_df.to_numpy().shape)
print(x_train_NV.shape)

In [ ]:
K = 5
        
cv_train_err_list = []
cv_valid_err_list = []
# pipeline = sklearn.pipeline.Pipeline(
#     steps = [('Log regr', LogisticRegression(C= 1e-10, penalty='l1', tol=1e-7))])

#y_train_df = np.array(y_train_df)
length = 2400
split = 0.8
split_spot = 1920
rand_indicies = np.random.permutation(length)
x_train_array = x_train_df.to_numpy()
x_train_text = x_train_array[rand_indicies]
x_train_NV = x_train_NV[rand_indicies]
y_train_df = y_train_df[rand_indicies]

x_train_split = x_train_NV[0:split_spot]
y_train_split = y_train_df[0:split_spot]
x_test_split = x_train_NV[split_spot:2400]
y_test_split = y_train_df[split_spot:2400]

pipeline = LogisticRegression(C= 0.1, penalty='l2', tol=1e-5)

pipeline.fit(x_train_split, y_train_split)

preds = pipeline.predict(x_test_split)
y_proba_preds = pipeline.predict_proba(x_test_split)

In [ ]:
false_positives = np.intersect1d(np.where(preds == 1)[0], np.where(y_test_split == 0)[0]) + 1920
false_negatives = np.intersect1d(np.where(preds == 0)[0], np.where(y_test_split == 1)[0]) + 1920
print(false_negatives)
# print(len(false_positives))
# print(x_train_df.shape)
# for idx in false_negatives:
#     print(x_train_df[idx])
for idx in false_negatives:
    print(x_train_text[idx])
    print(y_train_df[idx])
    print(preds[idx - 1920])

In [ ]:
# print(len(false_positives))
# print(x_train_df.shape)
# for idx in false_negatives:
#     print(x_train_df[idx])
for idx in false_positives:
    print(x_train_text[idx])

In [ ]:
# compute AUROC score
auroc_score = roc_auc_score( y_test_split, y_proba_preds[:,1]) 
print(auroc_score)

In [ ]:
print(cv_train_err_list)
print(cv_valid_err_list)
# for C= 1e-10, penalty='l2', tol=1e-7
#   train_err = 0.43073043
#   valid_err = 0.52891713
# for C= 1e6, penalty='l2', tol=1e-7
#   train_err = 0.07053015
#   valid_err = 0.48599428
# for C= 1, penalty='l2', tol=1e-3
#   train_err = 0.19003116
#   valid_err = 0.44722232
# for C= 10-5, penalty='l2', tol=1e-1
#   train_err = 0.12028722
#   valid_err = 0.4509112